# 🚀 **Módulo 2: Preparando os dados como um chef prepara ingredientes**

## **Aula 2.1: Formatos de dados para fine tuning**

---

### **Tá, mas por que os dados são importantes?**

Imagine que você é um chef e vai fazer um prato especial. Você pode ter as melhores panelas, o fogão mais moderno, mas se os ingredientes estiverem estragados ou na quantidade errada, o prato vai sair uma merda! 😅

**Fine Tuning é a mesma coisa!** Os dados são os "ingredientes" que vão ensinar sua IA. Se os dados forem ruins, a IA vai aprender coisas erradas.

**Por que preparar dados é crucial?**

É como ensinar uma criança:
- **Dados bons**: A criança aprende a falar corretamente
- **Dados ruins**: A criança aprende gírias e palavrões (ops!)

---

**🖼️ Sugestão de imagem**: Chef organizando ingredientes frescos vs ingredientes estragados

### **Setup Inicial - Preparando o Terreno**

In [ ]:
# Importações necessárias
import pandas as pd
import numpy as np
import json
from datasets import Dataset
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re

print("✅ Bibliotecas importadas com sucesso!")

## **1. Formatos de dados para Fine Tuning**

Vamos ver os formatos mais comuns:

In [ ]:
# Exemplos de formatos de dados
print("📊 FORMATOS DE DADOS PARA FINE TUNING\n")

# 1. Formato Conversacional (Chat)
dados_chat = [
    {
        "messages": [
            {"role": "user", "content": "Qual é o melhor iPhone para fotografia?"},
            {"role": "assistant", "content": "O iPhone 15 Pro Max é o melhor para fotografia, com câmera de 48MP e zoom óptico de 5x."}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "Quanto custa o iPhone 15?"},
            {"role": "assistant", "content": "O iPhone 15 custa R$ 6.999 no Brasil."}
        ]
    }
]

print("1️⃣ FORMATO CONVERSACIONAL (Chat):")
print(json.dumps(dados_chat[0], indent=2, ensure_ascii=False))
print()

# 2. Formato Instrução-Resposta
dados_instrucao = [
    {
        "instruction": "Explique a diferença entre iPhone 15 e iPhone 15 Pro",
        "input": "",
        "output": "O iPhone 15 Pro tem chip A17 Pro, câmera tripla, tela ProMotion e acabamento em titânio, enquanto o iPhone 15 tem chip A16, câmera dupla, tela normal e acabamento em alumínio."
    },
    {
        "instruction": "Calcule o preço com desconto",
        "input": "iPhone 15 custa R$ 6.999 com 10% de desconto",
        "output": "R$ 6.999 - 10% = R$ 6.299,10"
    }
]

print("2️⃣ FORMATO INSTRUÇÃO-RESPOSTA:")
print(json.dumps(dados_instrucao[0], indent=2, ensure_ascii=False))
print()

# 3. Formato Pergunta-Resposta
dados_qa = [
    {
        "question": "Qual é a capacidade de armazenamento do iPhone 15?",
        "answer": "O iPhone 15 está disponível em 128GB, 256GB e 512GB."
    },
    {
        "question": "O iPhone 15 tem carregamento sem fio?",
        "answer": "Sim, o iPhone 15 suporta carregamento sem fio MagSafe e Qi."
    }
]

print("3️⃣ FORMATO PERGUNTA-RESPOSTA:")
print(json.dumps(dados_qa[0], indent=2, ensure_ascii=False))
print()

print("💡 Cada formato tem seu uso específico. Vamos ver qual usar quando!")

## **2. Quando usar cada formato**

É como escolher o formato certo para cada tipo de comunicação:

In [ ]:
# Comparação dos formatos
formatos_comparacao = {
    "Conversacional": {
        "melhor_para": "Assistentes de chat, atendimento ao cliente",
        "vantagens": "Natural, mantém contexto",
        "desvantagens": "Mais complexo de preparar",
        "exemplo": "Chatbot de vendas"
    },
    "Instrução-Resposta": {
        "melhor_para": "Tarefas específicas, instruções claras",
        "vantagens": "Simples, direto",
        "desvantagens": "Menos natural",
        "exemplo": "Assistente de análise de dados"
    },
    "Pergunta-Resposta": {
        "melhor_para": "FAQ, conhecimento específico",
        "vantagens": "Fácil de preparar, claro",
        "desvantagens": "Limitado a Q&A",
        "exemplo": "FAQ de produtos"
    }
}

print("🎯 QUANDO USAR CADA FORMATO\n")

for formato, info in formatos_comparacao.items():
    print(f"🔹 {formato}")
    print(f"   🎯 Melhor para: {info['melhor_para']}")
    print(f"   ✅ Vantagens: {info['vantagens']}")
    print(f"   ⚠️  Desvantagens: {info['desvantagens']}")
    print(f"   💡 Exemplo: {info['exemplo']}")
    print()

## **3. Exemplo Prático: Criando dataset de vendas**

Vamos criar um dataset real para um assistente de vendas:

In [ ]:
# Criando dataset de vendas de iPhone
print("🛍️ CRIANDO DATASET DE VENDAS DE IPHONE\n")

# Dados dos produtos
produtos = {
    "iPhone 15": {
        "preco": "R$ 6.999",
        "cores": "Preto, Azul, Verde, Rosa, Amarelo",
        "capacidades": "128GB, 256GB, 512GB",
        "caracteristicas": "Chip A16, câmera dupla 48MP, tela 6.1",
        "publico": "Uso básico, fotografia casual"
    },
    "iPhone 15 Pro": {
        "preco": "R$ 8.999",
        "cores": "Titânio Natural, Titânio Azul, Titânio Branco, Titânio Preto",
        "capacidades": "128GB, 256GB, 512GB, 1TB",
        "caracteristicas": "Chip A17 Pro, câmera tripla 48MP, tela 6.1 ProMotion",
        "publico": "Profissionais, fotógrafos, gamers"
    },
    "iPhone 15 Pro Max": {
        "preco": "R$ 9.999",
        "cores": "Titânio Natural, Titânio Azul, Titânio Branco, Titânio Preto",
        "capacidades": "256GB, 512GB, 1TB",
        "caracteristicas": "Chip A17 Pro, câmera tripla 48MP, zoom 5x, tela 6.7 ProMotion",
        "publico": "Profissionais avançados, fotógrafos, criadores de conteúdo"
    }
}

# Criando dataset conversacional
dataset_vendas = []

# Perguntas sobre preços
perguntas_preco = [
    "Quanto custa o iPhone 15?",
    "Qual é o preço do iPhone 15 Pro?",
    "Quanto sai o iPhone 15 Pro Max?",
    "Qual é o iPhone mais barato?",
    "Qual é o iPhone mais caro?"
]

respostas_preco = [
    "O iPhone 15 custa R$ 6.999.",
    "O iPhone 15 Pro custa R$ 8.999.",
    "O iPhone 15 Pro Max custa R$ 9.999.",
    "O iPhone 15 é o mais barato, custando R$ 6.999.",
    "O iPhone 15 Pro Max é o mais caro, custando R$ 9.999."
]

for pergunta, resposta in zip(perguntas_preco, respostas_preco):
    dataset_vendas.append({
        "messages": [
            {"role": "user", "content": pergunta},
            {"role": "assistant", "content": resposta}
        ]
    })

# Perguntas sobre características
perguntas_caracteristicas = [
    "Qual iPhone tem a melhor câmera?",
    "Qual iPhone é melhor para jogos?",
    "Qual iPhone tem a maior tela?",
    "Qual iPhone tem zoom óptico?"
]

respostas_caracteristicas = [
    "O iPhone 15 Pro Max tem a melhor câmera, com zoom óptico de 5x e sensor de 48MP.",
    "O iPhone 15 Pro e Pro Max são melhores para jogos, com chip A17 Pro e tela ProMotion.",
    "O iPhone 15 Pro Max tem a maior tela, com 6.7 polegadas.",
    "O iPhone 15 Pro Max tem zoom óptico de 5x, o maior da linha."
]

for pergunta, resposta in zip(perguntas_caracteristicas, respostas_caracteristicas):
    dataset_vendas.append({
        "messages": [
            {"role": "user", "content": pergunta},
            {"role": "assistant", "content": resposta}
        ]
    })

print(f"✅ Dataset criado com {len(dataset_vendas)} exemplos")
print("\n📝 Exemplos do dataset:")

for i, exemplo in enumerate(dataset_vendas[:3], 1):
    print(f"\n{i}. Pergunta: {exemplo['messages'][0]['content']}")
    print(f"   Resposta: {exemplo['messages'][1]['content']}")

## **4. Limpeza e preparação de dados**

Agora vamos limpar e preparar os dados:

In [ ]:
# Funções de limpeza de dados
def limpar_texto(texto):
    """
    Limpa o texto removendo caracteres especiais e normalizando
    """
    # Remove caracteres especiais mas mantém acentos
    texto = re.sub(r'[^\w\s\-.,!?;:()]', '', texto)
    # Remove espaços extras
    texto = re.sub(r'\s+', ' ', texto)
    # Remove espaços no início e fim
    texto = texto.strip()
    return texto

def validar_exemplo(exemplo):
    """
    Valida se um exemplo está correto
    """
    # Verifica se tem a estrutura correta
    if 'messages' not in exemplo:
        return False
    
    if len(exemplo['messages']) != 2:
        return False
    
    # Verifica se tem user e assistant
    if exemplo['messages'][0]['role'] != 'user' or exemplo['messages'][1]['role'] != 'assistant':
        return False
    
    # Verifica se os textos não estão vazios
    if not exemplo['messages'][0]['content'].strip() or not exemplo['messages'][1]['content'].strip():
        return False
    
    return True

def preparar_dataset(dataset_bruto):
    """
    Prepara o dataset para treinamento
    """
    dataset_limpo = []
    
    for exemplo in dataset_bruto:
        if validar_exemplo(exemplo):
            # Limpa os textos
            exemplo_limpo = {
                "messages": [
                    {
                        "role": "user",
                        "content": limpar_texto(exemplo['messages'][0]['content'])
                    },
                    {
                        "role": "assistant",
                        "content": limpar_texto(exemplo['messages'][1]['content'])
                    }
                ]
            }
            dataset_limpo.append(exemplo_limpo)
    
    return dataset_limpo

# Aplicando limpeza
print("🧹 LIMPANDO E PREPARANDO DADOS\n")

dataset_limpo = preparar_dataset(dataset_vendas)

print(f"📊 Estatísticas do dataset:")
print(f"   Total de exemplos: {len(dataset_vendas)}")
print(f"   Exemplos válidos: {len(dataset_limpo)}")
print(f"   Taxa de validação: {len(dataset_limpo)/len(dataset_vendas)*100:.1f}%")

# Análise de comprimento dos textos
comprimentos_perguntas = [len(ex['messages'][0]['content']) for ex in dataset_limpo]
comprimentos_respostas = [len(ex['messages'][1]['content']) for ex in dataset_limpo]

print(f"\n📏 Comprimento dos textos:")
print(f"   Perguntas - Média: {np.mean(comprimentos_perguntas):.1f}, Min: {min(comprimentos_perguntas)}, Max: {max(comprimentos_perguntas)}")
print(f"   Respostas - Média: {np.mean(comprimentos_respostas):.1f}, Min: {min(comprimentos_respostas)}, Max: {max(comprimentos_respostas)}")

## **5. Visualizando a qualidade dos dados**

Vamos criar gráficos para entender nossos dados:

In [ ]:
# Criando visualizações
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# 1. Distribuição de comprimento das perguntas
ax1.hist(comprimentos_perguntas, bins=10, alpha=0.7, color='skyblue')
ax1.set_title('📏 Comprimento das Perguntas')
ax1.set_xlabel('Número de caracteres')
ax1.set_ylabel('Frequência')
ax1.axvline(np.mean(comprimentos_perguntas), color='red', linestyle='--', label=f'Média: {np.mean(comprimentos_perguntas):.1f}')
ax1.legend()

# 2. Distribuição de comprimento das respostas
ax2.hist(comprimentos_respostas, bins=10, alpha=0.7, color='lightgreen')
ax2.set_title('📏 Comprimento das Respostas')
ax2.set_xlabel('Número de caracteres')
ax2.set_ylabel('Frequência')
ax2.axvline(np.mean(comprimentos_respostas), color='red', linestyle='--', label=f'Média: {np.mean(comprimentos_respostas):.1f}')
ax2.legend()

# 3. Relação entre comprimento de pergunta e resposta
ax3.scatter(comprimentos_perguntas, comprimentos_respostas, alpha=0.6, color='orange')
ax3.set_title('📊 Pergunta vs Resposta')
ax3.set_xlabel('Comprimento da pergunta')
ax3.set_ylabel('Comprimento da resposta')

# 4. Palavras mais comuns nas perguntas
palavras_perguntas = []
for exemplo in dataset_limpo:
    palavras = exemplo['messages'][0]['content'].lower().split()
    palavras_perguntas.extend(palavras)

contador_palavras = Counter(palavras_perguntas)
palavras_comuns = contador_palavras.most_common(10)

palavras, contagens = zip(*palavras_comuns)
ax4.barh(range(len(palavras)), contagens, color='lightcoral')
ax4.set_yticks(range(len(palavras)))
ax4.set_yticklabels(palavras)
ax4.set_title('🔤 Palavras Mais Comuns nas Perguntas')
ax4.set_xlabel('Frequência')

plt.tight_layout()
plt.show()

print("💡 Estes gráficos nos ajudam a entender a qualidade e distribuição dos nossos dados!")

## **6. Salvando o dataset**

Agora vamos salvar nosso dataset limpo:

In [ ]:
# Salvando o dataset
print("💾 SALVANDO O DATASET\n")

# Salvando em JSON
with open('datasets/iphone_vendas.json', 'w', encoding='utf-8') as f:
    json.dump(dataset_limpo, f, ensure_ascii=False, indent=2)

print("✅ Dataset salvo em 'datasets/iphone_vendas.json'")

# Convertendo para formato Hugging Face Dataset
try:
    # Flattening o dataset para o formato do Hugging Face
    dataset_hf = []
    for exemplo in dataset_limpo:
        dataset_hf.append({
            'user': exemplo['messages'][0]['content'],
            'assistant': exemplo['messages'][1]['content']
        })
    
    # Criando Dataset do Hugging Face
    hf_dataset = Dataset.from_list(dataset_hf)
    
    # Salvando
    hf_dataset.save_to_disk('datasets/iphone_vendas_hf')
    
    print("✅ Dataset salvo em formato Hugging Face")
    print(f"📊 Informações do dataset:")
    print(f"   Número de exemplos: {len(hf_dataset)}")
    print(f"   Colunas: {hf_dataset.column_names}")
    
except Exception as e:
    print(f"⚠️  Erro ao salvar em formato Hugging Face: {e}")

print("\n🎉 Dataset preparado e salvo com sucesso!")

## **7. Teste Rápido**

Vamos testar seu entendimento sobre preparação de dados:

In [ ]:
# Teste rápido sobre preparação de dados
print("🧪 TESTE RÁPIDO - PREPARAÇÃO DE DADOS\n")

perguntas_teste = [
    {
        "pergunta": "Qual formato é melhor para um chatbot de atendimento?",
        "opcoes": ["A) Pergunta-Resposta", "B) Conversacional", "C) Instrução-Resposta"],
        "resposta": "B",
        "explicacao": "Formato conversacional é mais natural para chatbots!"
    },
    {
        "pergunta": "Por que é importante limpar os dados antes do fine tuning?",
        "opcoes": [
            "A) Para economizar espaço", 
            "B) Para melhorar a qualidade do treinamento", 
            "C) Para acelerar o download"
        ],
        "resposta": "B",
        "explicacao": "Dados limpos resultam em melhor qualidade de treinamento!"
    },
    {
        "pergunta": "Quantos exemplos são recomendados para fine tuning?",
        "opcoes": [
            "A) 10-50 exemplos", 
            "B) 100-1000 exemplos", 
            "C) 10.000+ exemplos"
        ],
        "resposta": "B",
        "explicacao": "100-1000 exemplos de qualidade são suficientes para começar!"
    }
]

for i, q in enumerate(perguntas_teste, 1):
    print(f"❓ {i}. {q['pergunta']}")
    for opcao in q['opcoes']:
        print(f"   {opcao}")
    print(f"💡 Resposta: {q['resposta']} - {q['explicacao']}")
    print()

print("🎉 Parabéns! Você já entende os fundamentos da preparação de dados!")

## **8. Desafio do Módulo**

Agora é sua vez de criar um dataset:

In [ ]:
# Desafio do módulo
print("🎯 DESAFIO DO MÓDULO\n")

print("Crie um dataset para um dos seguintes casos:")
print("\n1️⃣ Tutor de Matemática")
print("   - Perguntas sobre álgebra, geometria, cálculo")
print("   - Explicações passo a passo")
print("   - Exemplos práticos")

print("\n2️⃣ Analista de Sentimentos")
print("   - Comentários de clientes")
print("   - Classificação positiva/negativa/neutra")
print("   - Explicação do sentimento")

print("\n3️⃣ Assistente de Fitness")
print("   - Perguntas sobre exercícios")
print("   - Planos de treino")
print("   - Dicas de nutrição")

print("\n📝 Escolha um tema e crie pelo menos 10 exemplos no formato conversacional!")

## **🎉 Módulo 2 Concluído!**

### **O que aprendemos:**

✅ **Formatos de dados para Fine Tuning** (Conversacional, Instrução-Resposta, Pergunta-Resposta)  
✅ **Quando usar cada formato**  
✅ **Como limpar e preparar dados**  
✅ **Validação de qualidade dos dados**  
✅ **Visualização e análise de datasets**  
✅ **Salvamento em diferentes formatos**

### **Próximos Passos:**

🚀 **Módulo 3**: Escolhendo o modelo certo como escolher um carro  
🚀 **Módulo 4**: Treinando como um personal trainer  
🚀 **Módulo 5**: Avaliando como um professor corrige prova  
🚀 **Módulo 6**: Deploy como abrir um restaurante

---

**💡 Dica do Instrutor**: Dados são como ingredientes - quanto melhor a qualidade, melhor o resultado final! Agora que temos os "ingredientes" prontos, vamos escolher o "fogão" (modelo) certo! 😄

**🚀 Próximo módulo**: Vamos escolher o modelo base ideal para o nosso Fine Tuning!